In [43]:
# Generates elevator calls for people from a single flat
# TODO(vedensky): add types of flats
# TODO(vedensky): one person per flat works 09 - 18
def GenerateCallsForFlat(numPeople, floor):
    result = []
    for i in range(numPeople):
        time1 = random.randint(0, 22*60*60)
        result.append((time1, floor, 1))
        time2 = random.randint(time1 + 60*60, 24*60*60)
        result.append((time2, 1, floor))
    return result

In [44]:
# TODO(vedensky): make the number of people random
def GetPeopleArrivingAtFloors(numFloors, numFlatsPerFloor, avgPeoplePerFlat):
    result = []
    for floor in range(2, numFloors+1):
        for flat in range(numFlatsPerFloor):
            result.extend(GenerateCallsForFlat(avgPeoplePerFlat, floor))
    return result

In [45]:
import random
random.seed(4)

# Number of floors, including the ground floor
# Ground floor is 1, top floor is NumFloors
NumFloors = 10
# Exactly the same on each floor
NumFlatsPerFloor = 10
# Average number of people per flat - should be random
AvgPeoplePerFlat = 2

ElevatorCalls = sorted(GetPeopleArrivingAtFloors(NumFloors, NumFlatsPerFloor, AvgPeoplePerFlat))
#[(1235, 1, 3), (17983, 2, 1)]

ElevatorCalls.append((60*60*60, -1, -1))

print(len(ElevatorCalls))

361


In [46]:
#ElevatorCalls

In [47]:
#NumElevators = 1
SecondsBetweenFloors = 5
SecondsOnFloor = 10
MaxCapacity = 5

CurrentSecond = 0
CurrentCalls = []
ElevatorPosition = 1

ElevatorCallsIdx = 0

DebugLevel = 0

def GetCallsUpToCurrentSecond():
    global ElevatorCallsIdx
    result = []
    while (ElevatorCalls[ElevatorCallsIdx][0] <= CurrentSecond):
        result.append(ElevatorCalls[ElevatorCallsIdx])
        ElevatorCallsIdx += 1
    return result

def MoveElevatorToFloor(targetFloor):
    global CurrentSecond
    global ElevatorPosition
    if DebugLevel >= 1:
        print("Second: ", CurrentSecond, " - Elevator moved from floor ", ElevatorPosition, " to floor ", targetFloor)
    CurrentSecond += SecondsBetweenFloors*abs(targetFloor - ElevatorPosition)
    ElevatorPosition = targetFloor
    
def IsCalledFromCurrentFloor():
    for call in CurrentCalls:
        if call[1] == ElevatorPosition:
            return True
    return False

if DebugLevel >= 1:
    print(ElevatorCalls)
    print("Start: second 0, elevator is at the floor 1, empty.")
    
WaitTimes = []

while (ElevatorCallsIdx != len(ElevatorCalls) - 1) or (len(CurrentCalls) != 0):
    if len(CurrentCalls) == 0:
        CurrentSecond = ElevatorCalls[ElevatorCallsIdx][0]
    CurrentCalls.extend(GetCallsUpToCurrentSecond())
    if IsCalledFromCurrentFloor():
        CurrentSecond += SecondsOnFloor
        CurrentCalls.extend(GetCallsUpToCurrentSecond())
        MomentCalls = []
        for call in CurrentCalls:
            if call[1] == ElevatorPosition:
                MomentCalls.append(call)
                if len(MomentCalls) == MaxCapacity:
                    break
        if DebugLevel >= 1:
            print("Second: ", CurrentSecond, " - Elevator picked up: ", MomentCalls)
        maxFloor = 1
        setFloors = set()
        for call in MomentCalls:
            WaitTimes.append(CurrentSecond - call[0])
            setFloors.add(call[2])
            maxFloor = max(maxFloor, call[2])
            CurrentCalls.remove(call)
        if DebugLevel >= 1:
            if len(setFloors)-1 > 0:
                print("Second: ", CurrentSecond, " - Elevator unloads at ", len(setFloors)-1, " mid floors")
        MoveElevatorToFloor(maxFloor)
        CurrentSecond += (len(setFloors)-1)*SecondsOnFloor
        if DebugLevel >= 1:
            print("Second: ", CurrentSecond, " - Elevator unloads at floor ", ElevatorPosition)
        CurrentSecond += SecondsOnFloor
        CurrentCalls.extend(GetCallsUpToCurrentSecond())
        if (IsCalledFromCurrentFloor()):
            CurrentSecond -= SecondsOnFloor
        continue
    else:
        MoveElevatorToFloor(CurrentCalls[0][1])
        continue

avgWaitTime = 0
maxWaitTime = 0
for t in WaitTimes:
    avgWaitTime += t
    maxWaitTime = max(maxWaitTime, t)
print("Average wait time: ", avgWaitTime / len(WaitTimes))
print("Max wait time: ", maxWaitTime)

Average wait time:  35.541666666666664
Max wait time:  121
